In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import keras_tuner as kt
import tensorboard
from functools import partial

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.client import device_lib 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet101V2, Xception, InceptionResNetV2
from tensorflow.keras.applications import resnet_v2, xception, inception_resnet_v2
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical, Sequence

tf.random.set_seed(42)

global PROJECT_DIRECTORY
PROJECT_DIRECTORY = os.getcwd()

train_directory = "./data/organized/train/"
val_directory = "./data/organized/val/"
test_directory = "./data/organized/test/"

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
def inspect_tf_dataset(tf_dataset):
    for batch, labels in tf_dataset:
        print(f"Batch Shape: {batch.shape}")
        print(f"Labels Shape: {labels.shape}\n")
        print(f"type Batch:\n {batch.dtype}\n")
        print(f"type Labels:\n {labels.dtype}\n")
        print("\n=======================================")
        print(f"Feature Batch: {batch}")
        print(f"Labels: {labels}")
        print("=======================================\n")
        try:
            print(f"len(Batch):\n {len(batch)}\n")
            print(f"len(Labels):\n {len(labels)}\n")
        except:
            print("Can't call length on this!")
        
        
        for batch_num in range(1):
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
            print(f"Batch Item Number: {batch_num}")
            print(f"Batch[{batch_num}].shape:\n {batch[batch_num].shape}\n")
            print(f"Labels[{batch_num}].shape:\n {labels[batch_num].shape}\n")
            print(f"type Batch[{batch_num}].shape:\n {batch[batch_num].dtype}\n")
            print(f"type Labels[{batch_num}].shape:\n {labels[batch_num].dtype}\n")
            print(f"Batch[{batch_num}]:\n {batch[batch_num]}\n")
            print(f"Labels[{batch_num}]:\n {labels[batch_num]}\n")
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        break

### Reading images from a directory

In [4]:
def create_tensorflow_datasets(image_size, train_directory, val_directory, test_directory, batch_size=32):
    
    train_dataset = image_dataset_from_directory(directory = train_directory,
                                                 labels='inferred',
                                                 label_mode = 'int',
                                                 image_size=image_size,
                                                 batch_size=batch_size,
                                                 smart_resize=True)

    val_dataset = image_dataset_from_directory(directory = val_directory,
                                               labels='inferred',
                                               label_mode = 'int',
                                               image_size=image_size,
                                               batch_size=batch_size,
                                               smart_resize=True)

    test_dataset = image_dataset_from_directory(directory = test_directory,
                                                labels = "inferred",
                                                label_mode = "int",
                                                image_size=image_size,
                                                batch_size=batch_size,
                                                smart_resize=True)
    
    return train_dataset, val_dataset, test_dataset

In [5]:
train_dataset, val_dataset, test_dataset = create_tensorflow_datasets(image_size=(520, 520),
                                                                      train_directory=train_directory,
                                                                      val_directory=val_directory,
                                                                      test_directory=test_directory,
                                                                      batch_size=32)

Found 10520 files belonging to 196 classes.
Found 3234 files belonging to 196 classes.
Found 2431 files belonging to 196 classes.


### Calculating the output of a pretrained model

In [6]:
def calculate_base_model_outputs(tf_dataset, base_model_type, input_shape, output_pooling):
    
    valid_base_model_types = ['resnet101', 'xception', 'inception_resnet']
    
    if base_model_type not in valid_base_model_types:
        print("/n===========================================================")
        print("Invalid input for parameter base_model_type")
        print(f"Valid inputs are: {valid_base_model_types}")
        print("===========================================================\n")
        return -1
    
    target_labels = []
    model_output_features = []
    

    data_augmentation = keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal"),
                                          layers.experimental.preprocessing.RandomRotation(0.1),
                                          layers.experimental.preprocessing.RandomZoom(0.2)])
    
    
    if base_model_type == 'resnet101':
        
        base_model =  ResNet101V2(weights='imagenet',
                                  input_shape = input_shape,
                                  include_top=False,
                                  pooling = output_pooling)
            
        for images, labels in tf_dataset:
                
            augmented_images = data_augmentation(images)

            preprocessed_images = resnet_v2.preprocess_input(augmented_images)

            features = base_model.predict(preprocessed_images)

            model_output_features.append(features)

            target_labels.append(labels)

        return np.concatenate(model_output_features), np.concatenate(target_labels)
        
    elif base_model_type == 'xception':
        
        base_model = Xception(weights='imagenet',
                              input_shape=input_shape,
                              include_top=False,
                              pooling = output_pooling)
        
        for images, labels in tf_dataset:
            
            augmented_images = data_augmentation(images)
            
            preprocessed_images = xception.preprocess_input(augmented_images)
            
            features = base_model.predict(preprocessed_images)
            
            model_output_features.append(features)
            
            target_labels.append(labels)
            
        return np.concatenate(model_output_features), np.concatenate(target_labels)
    
    elif base_model_type == 'inception_resnet':
        
        base_model = InceptionResNetV2(weights='imagenet',
                                       input_shape=input_shape,
                                       include_top=False,
                                       pooling = output_pooling)
        
        for images, labels in tf_dataset:
            
            augmented_images = data_augmentation(images)
            
            preprocessed_images = inception_resnet_v2.preprocess_input(augmented_images)
            
            features = base_model.predict(preprocessed_images)
            
            model_output_features.append(features)
            
            target_labels.append(labels)
            
        return (np.concatenate(model_output_features), np.concatenate(target_labels))

### Functions for writing Numpy Arrays to TFRecord Files

In [7]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a floast_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
    array = tf.io.serialize_tensor(array)
    return array

def parse_single_image(image, label):
  
    #define the dictionary -- the structure -- of our single example
    data = {
        'dim_1' : _int64_feature(image.shape[0]),
        'dim_2' : _int64_feature(image.shape[1]),
        'dim_3' : _int64_feature(image.shape[2]),
        'raw_image' : _bytes_feature(serialize_array(image)),
        'label' : _int64_feature(label)
    }
    
    #create an Example, wrapping the single features
    out = tf.train.Example(features=tf.train.Features(feature=data))

    return out

def get_filename(base_model_type, input_shape, output_pooling, num_unique_epochs, current_epoch, dataset_type):
    
    global PROJECT_DIRECTORY
    
    # Directory that will contain all outputs from base_model_type (as many data augmented variations as we make) that are
    # preprocessed in the same way... (same starting image size and output pooling).
    base_dir_name = f"pretrained_model_output_features/{base_model_type}_pool{str(output_pooling)}_inShape_{str(input_shape)}"
    base_save_directory = os.path.join(PROJECT_DIRECTORY, base_dir_name)
    
    # Adding a unique folder inside the above directory that specifies if this is a train, val or test set
    full_dir_path = os.path.join(base_save_directory, f"{dataset_type}_unique_epoch_{current_epoch}_of_{num_unique_epochs}")
    
    os.makedirs(full_dir_path, exist_ok = True)
    
    filename = os.path.join(full_dir_path, f"{dataset_type}")
    
    return filename


def write_images_to_multiple_shards(images, labels, base_model_type, input_shape, output_pooling, num_unique_epochs, current_epoch,
                                    dataset_type, max_files:int=10):
    
    filename = get_filename(base_model_type = base_model_type,
                            input_shape = input_shape,
                            output_pooling = output_pooling,
                            num_unique_epochs = num_unique_epochs,
                            current_epoch = current_epoch,
                            dataset_type = dataset_type)

    #determine the number of shards (single TFRecord files) we need:
    splits = (len(images)//max_files) + 1 #determine how many tfr shards are needed
    
    if len(images) % max_files == 0:
        splits-=1
        
    print(f"\nUsing {splits} shard(s) for {len(images)} files, with up to {max_files} samples per shard")

    file_count = 0
    
    for shard_file_num in range(splits):
        
        current_shard_name = filename + f"_{shard_file_num + 1}_of_{splits}.tfrecords"
        
        
        options = tf.io.TFRecordOptions(compression_type="ZLIB")
        writer = tf.io.TFRecordWriter(current_shard_name, options=options)
        #writer = tf.io.TFRecordWriter(current_shard_name, tf.io.TFRecordOptions(compression_type = "GZIP"))

        current_shard_count = 0
        
        while current_shard_count < max_files: #as long as our shard is not full
            
            #get the index of the file that we want to parse now
            index = (shard_file_num * max_files) + current_shard_count
            
            if index == len(images): #when we have consumed the whole data, preempt generation
                break
            
            current_image = images[index]
            current_label = labels[index]
            
            output = parse_single_image(image = current_image, label = current_label)
            
            writer.write(output.SerializeToString())
            current_shard_count+=1
            file_count += 1
        
    writer.close()
    print("\n=======================================================================================")
    print(f"Wrote {file_count} elements to TFRecord")
    print(f"Base file path: {filename}")
    print("=======================================================================================\n")
    return

In [8]:
# tf_dataset, base_model_type, input_shape, output_pooling

def create_base_model_output_dirs(train_dataset, val_dataset, test_dataset, base_model_type, input_shape, output_pooling, num_unique_epochs,
                                  verbose = True):
    
    global_clock = time.time()
    
    dataset_types = ['train', 'val', 'test']
    datasets = [train_dataset, val_dataset, test_dataset]
    
    for epoch_num in range(1, num_unique_epochs + 1):
    
        for dataset, dset_type in zip(datasets, dataset_types):
            
            if verbose:
                start_time = time.time()
                print("\n=======================================================================================")
                print(f"Calculating pretrained {base_model_type} outputs for the {dset_type} dataset.")
                print(f"Processing epoch set {epoch_num } of {num_unique_epochs}")
                print(f"Output pooling setting: {output_pooling}")
                print(f"Size of images before {base_model_type} preprocessing: {input_shape}")
                print("=======================================================================================\n")
            
            output_features, labels = calculate_base_model_outputs(tf_dataset = dataset,
                                                                   base_model_type = base_model_type,
                                                                   input_shape = input_shape,
                                                                   output_pooling = output_pooling)
            
            if verbose:
                print("\n=======================================================================================")
                print(f"Finished calcuating {base_model_type} outputs!")
                print(f"{base_model_type} {dset_type} outputs shape: {output_features.shape}")
                print(f"Time spent getting features: {time.time() - start_time} seconds.")
                print("Starting to save outputs to tfrRecord file...")
                save_start_time = time.time()
                print("=======================================================================================\n")
            
            write_images_to_multiple_shards(images = output_features,
                                            labels = labels,
                                            base_model_type = base_model_type,
                                            input_shape = input_shape,
                                            output_pooling = output_pooling,
                                            num_unique_epochs = num_unique_epochs,
                                            current_epoch = epoch_num,
                                            dataset_type = dset_type)
            
            if verbose:
                print("\n=======================================================================================")
                print("Finished saving to disk!")
                print(f"Time spent saving: {time.time() - save_start_time} seconds.")
                print("=======================================================================================\n")
            
            del output_features
            del labels
                
    if verbose:
        print("\n=======================================================================================")
        print("Finished getting preprocessed features for all datasets and epochs!")
        print(f"Time spent: {round(((time.time() - global_clock) / 60), 2)} minutes.")
        print("=======================================================================================\n")
    

In [9]:
create_base_model_output_dirs(train_dataset = train_dataset,
                              val_dataset = val_dataset,
                              test_dataset = test_dataset,
                              base_model_type = 'resnet101',
                              input_shape = (520, 520, 3),
                              output_pooling = 'None',
                              num_unique_epochs = 1,
                              verbose = True)


Calculating pretrained resnet101 outputs for the train dataset.
Processing epoch set 1 of 1
Output pooling setting: None
Size of images before resnet101 preprocessing: (520, 520, 3)


Finished calcuating resnet101 outputs!
resnet101 train outputs shape: (10520, 17, 17, 2048)
Time spent getting features: 383.49495458602905 seconds.
Starting to save outputs to tfrRecord file...


Using 1052 shard(s) for 10520 files, with up to 10 samples per shard

Wrote 10520 elements to TFRecord
Base file path: C:\Users\Braden\Desktop\Data_Science\04_General_Assembly\05_Projects\03_car\pretrained_model_output_features/resnet101_poolNone_inShape_(520, 520, 3)\train_unique_epoch_1_of_1\train


Finished saving to disk!
Time spent saving: 350.1042950153351 seconds.


Calculating pretrained resnet101 outputs for the val dataset.
Processing epoch set 1 of 1
Output pooling setting: None
Size of images before resnet101 preprocessing: (520, 520, 3)


Finished calcuating resnet101 outputs!
resnet101 val outputs

## Reading TFRecord Dataset back in

In [10]:
def get_all_tf_shard_paths(base_model_type, input_shape, output_pooling, epoch_num, dataset_type):
    
    global PROJECT_DIRECTORY
    
    valid_dataset_types = ['train', 'val', 'test']
    
    if dataset_type not in valid_dataset_types:
        print("/n===========================================================")
        print("Invalid input for parameter dataset_type")
        print(f"Valid inputs are: {valid_dataset_types}")
        print("===========================================================\n")
        return -1
    
    
    #base_dir_name = f"pretrained_model_output_features/{base_model_type}_pool{str(output_pooling)}_inShape_{str(input_shape)}"
    base_outputs_directory = os.path.join(os.path.join(PROJECT_DIRECTORY, "pretrained_model_output_features"),
                                          f"{base_model_type}_pool{str(output_pooling)}_inShape_{str(input_shape)}")
    
    
    #base_outputs_directory = f"./pretrained_model_output_features/{base_model_type}_pool{str(output_pooling)}_inShape_{str(input_shape)}/"
    all_dirs = os.listdir(base_outputs_directory)
    
    correct_dir = [directory for directory in all_dirs if (dataset_type in directory) and (f'epoch_{epoch_num}' in directory)]
        
    if len(correct_dir) != 1:
        print("/n============================ Error ===============================")
        print("Invalid directory filtering!")
        print(f"Filtering returned: {correct_dir}")
        return -1
        print("=========================================================================\n")
            
    correct_dir = correct_dir[0]
        
    filepath = os.path.join(base_outputs_directory, correct_dir)
    
    files = os.listdir(filepath)
    
    full_file_paths = [os.path.join(filepath, file) for file in files]
    
    return full_file_paths

In [11]:
def parse_tfrecords(example):
    
    feature_description = {
        "dim_1": tf.io.FixedLenFeature([], tf.int64),
        "dim_2": tf.io.FixedLenFeature([], tf.int64),
        "dim_3": tf.io.FixedLenFeature([], tf.int64),
        "raw_image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    
    example = tf.io.parse_single_example(example, feature_description)
    
    dim_1 = example['dim_1']
    dim_2 = example['dim_2']
    dim_3 = example['dim_3']
    raw_image = example['raw_image']
    label = example['label']
    
    feature = tf.io.parse_tensor(raw_image, out_type=tf.float32)
    feature = tf.reshape(feature, shape=[dim_1, dim_2, dim_3])
    
    
    return (feature, label)

In [12]:
def get_dataset_from_multiple_tfrecords(base_model_type, dataset_type, batch_size, input_shape, output_pooling, epoch_num, compression = "ZLIB"):
    
    filepaths = get_all_tf_shard_paths(base_model_type = base_model_type,
                                     input_shape = input_shape,
                                     output_pooling = output_pooling,
                                     epoch_num = epoch_num,
                                     dataset_type = dataset_type)
    
    print(f"length of file paths: {len(filepaths)}")
    print(f"first: {filepaths[0]}")
    print(f"last: {filepaths[-1]}")
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    #create the dataset
    dataset = tf.data.TFRecordDataset(filepaths, compression_type = compression,num_parallel_reads=AUTOTUNE)

    #pass every single feature through our mapping function
    dataset = dataset.map(parse_tfrecords).shuffle(batch_size * 10).batch(32).prefetch(AUTOTUNE)
    
    return dataset

In [13]:
train_big = get_dataset_from_multiple_tfrecords(base_model_type = 'resnet101',
                                                dataset_type = 'train',
                                                batch_size = 32,
                                                input_shape = (520, 520, 3),
                                                output_pooling = None,
                                                epoch_num = 1)

length of file paths: 1052
first: C:\Users\Braden\Desktop\Data_Science\04_General_Assembly\05_Projects\03_car\pretrained_model_output_features\resnet101_poolNone_inShape_(520, 520, 3)\train_unique_epoch_1_of_1\train_1000_of_1052.tfrecords
last: C:\Users\Braden\Desktop\Data_Science\04_General_Assembly\05_Projects\03_car\pretrained_model_output_features\resnet101_poolNone_inShape_(520, 520, 3)\train_unique_epoch_1_of_1\train_9_of_1052.tfrecords


In [14]:
inspect_tf_dataset(train_big)

Batch Shape: (32, 17, 17, 2048)
Labels Shape: (32,)

type Batch:
 <dtype: 'float32'>

type Labels:
 <dtype: 'int64'>


Feature Batch: [[[[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        

In [38]:
type(train_big)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [20]:
os.getcwd()

'C:\\Users\\Braden\\Desktop\\Data_Science\\04_General_Assembly\\05_Projects\\03_car'

In [40]:
train_dataset = get_tf_dataset(base_model_type = "resnet101",
                               dataset_type = "train",
                               batch_size = 32,
                               input_shape = (520, 520, 3),
                               output_pooling = None,
                               epoch_num = 1)

In [ ]:
'''
train_dataset = get_tf_dataset(base_model_type = "resnet101",
                               dataset_type = "train",
                               batch_size = 32,
                               input_shape = (520, 520, 3),
                               output_pooling = None,
                               epoch_num = 1)''';

In [19]:
'''
import glob

train_files = sorted(glob.glob('./train*.tfrecord'))
for f_i, file in enumerate(train_files): 
    print(f_i) 
    total_images += sum([1 for _ in tf.python_io.tf_record_iterator(file)])''';

[]